In [ ]:
import pandas.io.sql as sql
from lxml.html import parse
from urllib.request import urlopen
import pandas as pd
import urllib 
import numpy as np
from matplotlib.pyplot import plot,savefig 
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import statsmodels.api as sm
import scipy.stats as stats 

In [ ]:
def get_sina_stock(stockid,start="1990-12-19",end=pd.datetime.now(),isstock=True,fuquan=True):    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content().strip() for val in elts] # .strip()鍘绘帀\r\t\n涔嬬被鐨勫瓧绗�    
    def parse_options_data(table,isstock,fuquan):
        rows = table.findall('.//tr')
        data = [_unpack(r) for r in rows[2:]]
        if isstock & fuquan:
            colnames = ['date','open','high','close','low','vol','amount','ratio'] # 鍥為伩涓枃澶勭悊
        else:
            colnames = ['date','open','high','close','low','vol','amount']
        data= pd.DataFrame(data,columns=colnames)
        data.index=pd.to_datetime(data["date"])
        data=data.drop("date",axis=1)
        data=data.ix[range(len(data)-1,-1,-1)]
        return  data
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="Q")
    daterange=daterange.insert(len(daterange),daterange[-1]+1)
    for cq in daterange: 
        if isstock:
            if fuquan:
                url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
            else:
                url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)            
        else:            
            url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'/type/S.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
        try:
            parsed = parse(urlopen(url))
        except:
            print ("download failed for year=" +str(cq.year)+',  jidu='+str(cq.month/3))
            continue
        doc = parsed.getroot()
        tables=doc.findall('.//table')     
        da= tables[-1] # last table 19 for fuquan data and 4 for index
        datatem = parse_options_data(da,isstock,fuquan)
        data=pd.concat([data,datatem])
    return pd.DataFrame(data[start:end],dtype=float)    

In [ ]:
#1.计算每只股票的月收益率，选择若干只(>=小组人数)分析其分布。
data1=get_sina_stock('600007','1991-1-1','2016-11-16').close
data2=get_sina_stock('600048','1991-1-1','2016-11-16').close
data3=get_sina_stock('600052','1991-1-1','2016-11-16').close
data4=get_sina_stock('600053','1991-1-1','2016-11-16').close
data1.name='600007'#中国国贸
data2.name='600048'#保利地产
data3.name='600052'#浙江广厦
data4.name='600053'#九鼎投资
data=pd.concat([data1,data2,data3,data4],axis=1)

In [ ]:
rtn_m=data.resample('M',how='last').pct_change()
rtn_m.describe()
print(rtn_m)

In [ ]:
####################################################################################################################
#2.计算每段时间内行业内每只股票的月期望收益率和标准差,画两个图形，分别给出收益率和标准差的走势图形并作简单说明。   #
#要求：横轴为时间，纵轴为预期收益率或标准差，每只股票用一种颜色的线条表示，并将所有股票对应的线条画在一个坐标系内。#
####################################################################################################################
Emu = DataFrame()
Emu[0] = rtn_m['1991':'1994'].mean()
Emu[1] = rtn_m['1995':'1998'].mean()
Emu[2] = rtn_m['1999':'2002'].mean()
Emu[3] = rtn_m['2003':'2006'].mean()
Emu[4] = rtn_m['2007':'2010'].mean()
Emu[5] = rtn_m['2011':'2014'].mean()
Emu = Emu.transpose()
Emu.index = ['1991-1994','1995-1998','1999-2002','2003-2006','2007-2010','2011-2014']
Emu.plot()

Esigma = DataFrame()
Esigma[0] = rtn_m['1991':'1994'].std()
Esigma[1] = rtn_m['1995':'1998'].std()
Esigma[2] = rtn_m['1999':'2002'].std()
Esigma[3] = rtn_m['2003':'2006'].std()
Esigma[4] = rtn_m['2007':'2010'].std()
Esigma[5] = rtn_m['2011':'2014'].std()
Esigma = Esigma.transpose()
Esigma.index = ['1991-1994','1995-1998','1999-2002','2003-2006','2007-2010','2011-2014']
Esigma.plot()

plt.show()

In [ ]:
#3.以标准差为横轴，期望收益率和纵轴，用上面计算的所有数据作散点图，不同股票用不同颜色，同一支股票的不同时间得到的数据用相同的颜色。
plt.figure(1)
for i in range(len(stock_list)):
    list1 = Emu[stock_list[i]].tolist() 
    list2 = Esigma[stock_list[i]].tolist()
    plt.figure(1)
    plt.plot(list2,list1,'o')
    plt.title('scatter')
plt.show()

In [ ]:
#############################################################################################
#4.从上面股票中，对每个时间窗，计算股票收益率之间的相关系数矩阵，只取下三角形中不是1的数据。#
#  统计每个时间段相关系数的最大值，最小值，均值，25%和75%分为点（表格汇总）。               #
#  用box图画出相关系数的分布，横坐为时间轴，每个时间段对应一个分布图                        #
#############################################################################################

In [ ]:
######################################################
#进一步分析上面得到的结果，讨论如下问题：            #
#(a). 总体上看，高收益，高风险的模式是否成立？       #
#(b). 时间周轴上看，每只股票的收益风险特性是否稳定？ #
#(c). 相关系数的分布是否稳定？                       #
######################################################


In [ ]:
######################################################################
#5.	选择3只数据比较完整的股票，对每个时间段作投资组合分析，包括：    #
#(d). 在没有无风险资产的情况下计算并画出有效前沿                     #
#(e). 在有无风险资产的情形下计算并画出有效前沿，其中需要计算切线组合 #
#(f). 比较不同时间段计算的投资组合的差异，有效前沿，切线组合等       #
######################################################################
